# Функции и tf-idf

Но сначала установим нужное.

In [1]:
! pip install pymorphy2  # устанавливаем

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.5/55.5 KB 1.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 36.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13721 sha256=8700577957ea7972482dc7e210c98ff4f307697cec88299f9eb64ffd8ce62cef
  Stored in directory: /root/.cache/pip/wheels/70/4a/46/1309fc853b8d395e60bafaf1b6df7845bdd82c95fd59dd8d2b
Successfully built docopt


In [2]:
from pymorphy2 import MorphAnalyzer
morph = MorphAnalyzer()

In [3]:
! pip install razdel

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [4]:
from razdel import tokenize

In [5]:
import nltk
nltk.download('stopwords')
stopwords = nltk.corpus.stopwords.words('russian')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [6]:
stopwords[:10]

['и', 'в', 'во', 'не', 'что', 'он', 'на', 'я', 'с', 'со']

In [7]:
import os  # для работы с папками и файлами

In [8]:
import collections

# Знакомство с функциями

(чтобы потом проще было код примеров понимать)

Общий вид:

```
def <название_функции>(): 
    <тело/инструкции_функции>
    return <результат>
```

In [9]:
def say_hello():
    print('Привет!')

In [10]:
say_hello()

Привет!


In [11]:
def say_hello():
    username = input('Как тебя зовут? ')
    print(f'Привет, {username}!')

In [12]:
say_hello()

Как тебя зовут? Таня
Привет, Таня!


В скобках — входные параметры.

In [25]:
def say_hello(username):
    stroka = f'Привет, {username}!'
    print(stroka)

In [26]:
say_hello('Маша')

Привет, Маша!


In [27]:
# Если возраст > 26, "Здравствуйте", а если меньше - "Привет"
def say_hello(username, age):
    if age > 30:
        print(f'Здравствуйте, {username}!')
    else:
        print(f'Привет, {username}!')

In [28]:
say_hello('Рита', 24)

Привет, Рита!


### Тренировка

In [21]:
# Напишем функцию нахождения среднего
def mean(mylist):
    average = sum(mylist)/len(mylist)
    print(average)

In [22]:
mean([1, 2, 3, 4, 5])

3.0


In [23]:
another_list = [1, 2, 3.5, 4.1, 5]
mean(another_list)

3.12


In [24]:
gotten_average = mean([1, 2, 3])
print(gotten_average)

2.0
None


In [29]:
gotten_average = mean([1, 2, 3])
print('Что лежит в gotten_average?')
print(gotten_average)

2.0
Что лежит в gotten_average?
None


In [30]:
print(gotten_average)

None


В чём дело? Функция ничего не возвращает (она возвращает None).

return — верни

In [31]:
def mean(smthing):
    average = sum(smthing) / len(smthing)
    return average

In [32]:
gotten_average = mean([1, 2, 3])
print('Что лежит в gotten_average?')
print(gotten_average)

Что лежит в gotten_average?
2.0


Важно! То, что происходит внутри функции, остаётся внутри функции.

In [33]:
# Так не получится
print(average)

NameError: ignored

Любые переменные, объявленные внутри функции, для основного кода программы не существуют.

Это хорошо. Нам совершенно не важно, какие переменные использовали внутри функции `len()`, или `sum()`, или любой другой. Мы ничего случайно не переназовём и не испортим.

Могут быть несколько разных `return` в зависимости от условия


In [34]:
def mean(mylist):
    if type(mylist) == str:
        print('Осторожно, вы дали строку. У строки нельзя найти среднее. Функция возращает ту же самую строку.')
        return mylist
    else:
        average = sum(mylist) / len(mylist)
        return average

In [35]:
mean([1, 2, 3, 4, 5])

3.0

In [36]:
mean('kjnwf')

Осторожно, вы дали строку. У строки нельзя найти среднее. Функция возращает ту же самую строку.


'kjnwf'

# Про несколько текстов

Как сравнить? Как понять, какой текст о чём?  Как выделить ключевые слова?

In [37]:
file_list = os.listdir()  # список файлов и папок в директории, где запущена программа
file_list

['.config', 'smeshariki', '.ipynb_checkpoints', 'sample_data']

In [38]:
file_list = os.listdir('./smeshariki')
file_list

['1_skamejka.txt',
 '19_denrozhdenijanushi.txt',
 '17_la.txt',
 '8_ktopervyj.txt',
 '16_biografijazontika.txt',
 '4_fanernoesolnce.txt',
 '12_kaksobratdruzej.txt',
 '215_kaketobylo.txt']

In [39]:
text_lex = {}
for textname in file_list:
    with open('./smeshariki/'+textname, encoding='UTF-8') as f:
        text = f.read()
    tokens_with_boundaries = list(tokenize(text))
    tokens = [_.text for _ in tokens_with_boundaries]
    words = [w.lower() for w in tokens if w.isalpha()]
    parsed_words = []
    for w in words:
        parsed_words.append(morph.parse(w)[0])
    lex = []
    for word in parsed_words:
        lex.append(word.normal_form)
    text_lex[textname[:-4]] = lex  # получили словарь {название текста: список лемм}

In [40]:
list(text_lex.keys())

['1_skamejka',
 '19_denrozhdenijanushi',
 '17_la',
 '8_ktopervyj',
 '16_biografijazontika',
 '4_fanernoesolnce',
 '12_kaksobratdruzej',
 '215_kaketobylo']

In [41]:
text_lex['1_skamejka'][:20]

['крош',
 'давно',
 'хотеть',
 'сделать',
 'для',
 'свой',
 'дом',
 'полезный',
 'и',
 'решить',
 'смастерить',
 'скамейка',
 'после',
 'несколько',
 'день',
 'работа',
 'скамейка',
 'быть',
 'готовый',
 'крош']

- сравнить самые частотные слова

In [42]:
for text in text_lex:
    print(text)
    fdist = collections.Counter(text_lex[text])
    print(fdist.most_common(15))

1_skamejka
[('и', 21), ('он', 13), ('крош', 10), ('ёжик', 10), ('не', 10), ('бараш', 10), ('что', 8), ('скамейка', 7), ('на', 7), ('розовый', 6), ('но', 6), ('друг', 6), ('краска', 5), ('в', 5), ('вдохновение', 5)]
19_denrozhdenijanushi
[('на', 6), ('они', 5), ('кактус', 4), ('и', 3), ('ёжик', 3), ('друг', 3), ('день', 2), ('рождение', 2), ('всё', 2), ('подарок', 2), ('крош', 2), ('а', 2), ('нюша', 2), ('идти', 2), ('в', 2)]
17_la
[('и', 17), ('бараш', 13), ('в', 12), ('он', 9), ('на', 6), ('что', 6), ('колодец', 6), ('ля', 5), ('это', 5), ('ёжик', 5), ('всё', 4), ('наш', 4), ('к', 4), ('крош', 4), ('ведро', 4)]
8_ktopervyj
[('и', 30), ('нюша', 24), ('бараш', 19), ('что', 16), ('в', 13), ('он', 11), ('на', 10), ('с', 10), ('она', 9), ('но', 9), ('ёжик', 9), ('крош', 8), ('а', 7), ('дверь', 6), ('у', 5)]
16_biografijazontika
[('и', 16), ('зонтик', 15), ('в', 13), ('бараш', 12), ('он', 10), ('с', 9), ('они', 8), ('свой', 7), ('крош', 7), ('ёжик', 7), ('к', 6), ('тот', 5), ('это', 5), ('ч

Ну вот, стопслова не убрали. Убираем.

In [43]:
text_lex_nostopw = {}
for textname in text_lex:
    words_nostopw = []
    for word in text_lex[textname]:
        if word not in stopwords:
            words_nostopw.append(word)
    text_lex_nostopw[textname] = words_nostopw      

In [44]:
for text in text_lex_nostopw:
    print(text)
    fdist = collections.Counter(text_lex_nostopw[text])
    print(fdist.most_common(15))

1_skamejka
[('крош', 10), ('ёжик', 10), ('бараш', 10), ('скамейка', 7), ('розовый', 6), ('друг', 6), ('краска', 5), ('вдохновение', 5), ('говорить', 4), ('мочь', 4), ('понимать', 4), ('сделать', 3), ('дом', 3), ('приходить', 3), ('очень', 3)]
19_denrozhdenijanushi
[('кактус', 4), ('ёжик', 3), ('друг', 3), ('день', 2), ('рождение', 2), ('всё', 2), ('подарок', 2), ('крош', 2), ('нюша', 2), ('идти', 2), ('путь', 2), ('который', 2), ('нос', 1), ('нюшина', 1), ('c', 1)]
17_la
[('бараш', 13), ('колодец', 6), ('ля', 5), ('это', 5), ('ёжик', 5), ('всё', 4), ('наш', 4), ('крош', 4), ('ведро', 4), ('лампа', 4), ('кричать', 3), ('голос', 3), ('падать', 3), ('время', 3), ('нота', 2)]
8_ktopervyj
[('нюша', 24), ('бараш', 19), ('ёжик', 9), ('крош', 8), ('дверь', 6), ('это', 5), ('время', 4), ('друг', 4), ('видеть', 3), ('первый', 3), ('начинать', 3), ('открывать', 3), ('виноватый', 3), ('ссора', 3), ('говорить', 3)]
16_biografijazontika
[('зонтик', 15), ('бараш', 12), ('свой', 7), ('крош', 7), ('ёжи

- а ещё хорошо бы биграммы и триграммы посмотреть (больше совпадений - тексты ближе)

In [45]:
# Но не с нашими текстами. Они слишком маленькие.
bigr_l = list(nltk.bigrams(text_lex_nostopw['1_skamejka']))
bigr_counter = collections.Counter(bigr_l)
bigr_counter.most_common(10)

[(('крош', 'ёжик'), 4),
 (('розовый', 'краска'), 3),
 (('понимать', 'бараш'), 2),
 (('крош', 'давно'), 1),
 (('давно', 'хотеть'), 1),
 (('хотеть', 'сделать'), 1),
 (('сделать', 'свой'), 1),
 (('свой', 'дом'), 1),
 (('дом', 'полезный'), 1),
 (('полезный', 'решить'), 1)]

А если все тексты о Кроше, Ёжике и Бараше? Тогда все они будут похожими, потому что везде частотны слова "крош", "ёжик", "бараш"?

Да, но нам важно понимать, чем они отличаются.

Точнее будет смотреть на слова, которые в отдельных текстах частотные, а при этом в коллекции документов редкие.

**Есть метод! TF-IDF**

- TF — term frequency - частота слова внутри документа
- IDF — inverse document frequency - log(кол-во документов/кол-во документов с этим словом)

TF-IDF = TF*IDF

Чем больше TF-IDF, тем важнее слово.

\#(логарифм натуральный, по основанию e≈2,72, но можно и другой)

In [46]:
import pandas as pd

Давайте сначала на 3х коротких текстах

In [47]:
documentA = 'мама мыла раму'
documentB = 'Мила мыла куклу и раму'
documentC = 'Римма мыла раму и мама мыла раму'

In [48]:
bagOfWordsA = documentA.split(' ')
bagOfWordsB = documentB.split(' ')
bagOfWordsC = documentC.split(' ')

In [49]:
uniqueWords = set(bagOfWordsA + bagOfWordsB + bagOfWordsC)

In [50]:
# Вот все слова, которые встретились в текстах
uniqueWords

{'Мила', 'Римма', 'и', 'куклу', 'мама', 'мыла', 'раму'}

Делаем словарь {слово: кол-во употреблений} для каждого текста.


In [53]:
numOfWordsA = dict.fromkeys(uniqueWords, 0)
for word in bagOfWordsA:
    numOfWordsA[word] += 1
numOfWordsB = dict.fromkeys(uniqueWords, 0)
for word in bagOfWordsB:
    numOfWordsB[word] += 1
numOfWordsC = dict.fromkeys(uniqueWords, 0)
for word in bagOfWordsC:
    numOfWordsC[word] += 1

In [54]:
# В первом тексте столько раз встретилось каждое из слов
numOfWordsA

{'куклу': 0, 'мыла': 1, 'Мила': 0, 'Римма': 0, 'и': 0, 'раму': 1, 'мама': 1}

Считаем **TF** (частота слова внутри документа)

In [55]:
def computeTF(wordDict, bagOfWords):
    tfDict = {}
    bagOfWordsCount = len(bagOfWords)
    for word, count in wordDict.items():
        tfDict[word] = count / float(bagOfWordsCount)
    return tfDict

In [56]:
tfA = computeTF(numOfWordsA, bagOfWordsA)
tfB = computeTF(numOfWordsB, bagOfWordsB)
tfC = computeTF(numOfWordsC, bagOfWordsC)

In [57]:
tfA

{'куклу': 0.0,
 'мыла': 0.3333333333333333,
 'Мила': 0.0,
 'Римма': 0.0,
 'и': 0.0,
 'раму': 0.3333333333333333,
 'мама': 0.3333333333333333}

Считаем **IDF** (log(кол-во документов/кол-во документов с этим словом))

один раз для всех слов

In [61]:
def computeIDF(documents):
    import math
    N = len(documents)
    
    idfDict = dict.fromkeys(documents[0].keys(), 0)
    for document in documents:
        for word, val in document.items():
            if val > 0:
                idfDict[word] += 1
    
    for word, val in idfDict.items():
        idfDict[word] = math.log(N / float(val))
    return idfDict

In [62]:
idfs = computeIDF([numOfWordsA, numOfWordsB, numOfWordsC])

In [64]:
# Есть слова, которые встретились во многих текстах (значение меньше),
# есть "специфичные", которые встретились в малом кол-ве текстов (значение больше)
idfs

{'куклу': 1.0986122886681098,
 'мыла': 0.0,
 'Мила': 1.0986122886681098,
 'Римма': 1.0986122886681098,
 'и': 0.4054651081081644,
 'раму': 0.0,
 'мама': 0.4054651081081644}

In [65]:
def computeTFIDF(tfBagOfWords, idfs):
    tfidf = {}
    for word, val in tfBagOfWords.items():
        tfidf[word] = val * idfs[word]
    return tfidf

In [67]:
tfidfA = computeTFIDF(tfA, idfs)
tfidfB = computeTFIDF(tfB, idfs)
tfidfC = computeTFIDF(tfC, idfs)

df = pd.DataFrame([tfidfA, tfidfB, tfidfC])  # создаём таблицу

In [68]:
print('\n', documentA, '\n', documentB, '\n', documentC)
df


 мама мыла раму 
 Мила мыла куклу и раму 
 Римма мыла раму и мама мыла раму


,куклу,мыла,Мила,Римма,и,раму,мама
0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.135155
1,0.219722,0.0,0.219722,0.000000,0.081093,0.0,0.000000
2,0.000000,0.0,0.000000,0.156945,0.057924,0.0,0.057924


Чем больше значение, тем важнее слово для текста.

"Рама" встречается во всех трёх - у неё значение везде ноль.

"Кукла" и "Мила" встречаются только во втором - они очень важны для второго текста (у них самые большие значения)

### А теперь для смешариков то же самое


In [69]:
bagOfWords_001 = text_lex['1_skamejka']
bagOfWords_215 = text_lex['215_kaketobylo']
bagOfWords_016 = text_lex['16_biografijazontika']

In [70]:
kikoriki_uniqueW = set(bagOfWords_001 + bagOfWords_215 + bagOfWords_016)

In [71]:
numOfWords_001 = dict.fromkeys(kikoriki_uniqueW, 0)
for word in bagOfWords_001:
    numOfWords_001[word] += 1
numOfWords_215 = dict.fromkeys(kikoriki_uniqueW, 0)
for word in bagOfWords_215:
    numOfWords_215[word] += 1
numOfWords_016 = dict.fromkeys(kikoriki_uniqueW, 0)
for word in bagOfWords_016:
    numOfWords_016[word] += 1

In [72]:
idfs = computeIDF([numOfWords_001, numOfWords_215, numOfWords_016])

In [73]:
tfidfA = computeTFIDF(computeTF(numOfWords_001, bagOfWords_001), idfs)
tfidfB = computeTFIDF(computeTF(numOfWords_215, bagOfWords_215), idfs)
tfidfC = computeTFIDF(computeTF(numOfWords_016, bagOfWords_016), idfs)

df = pd.DataFrame([tfidfA, tfidfB, tfidfC])

In [74]:
df

,ошибочно,друг,чуть,розовый,вспоминать,краситься,вода,опечалить,выбивать,бревно,...,забираться,поедать,писать,начинаться,появляться,смастерить,забыть,подхватывать,до,итог
0,0.000000,0.007093,0.000000,0.019218,0.000000,0.003203,0.000000,0.003203,0.000000,0.000000,...,0.003203,0.000000,0.006406,0.000000,0.000000,0.003203,0.000000,0.000000,0.001182,0.003203
1,0.009079,0.000000,0.009079,0.000000,0.018159,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.009079,0.000000,0.000000,0.000000,0.000000,0.009079,0.000000,0.000000,0.000000
2,0.000000,0.005283,0.000000,0.000000,0.000000,0.000000,0.010736,0.000000,0.003579,0.003579,...,0.000000,0.000000,0.000000,0.007157,0.010736,0.000000,0.000000,0.007157,0.001321,0.000000


In [75]:
# Посмотрим на слова с наибольшим tf-idf в серии "Скамейка"
collections.Counter(tfidfA).most_common(15)

[('розовый', 0.019217707673494632),
 ('краска', 0.01601475639457886),
 ('вдохновение', 0.01601475639457886),
 ('понимать', 0.012811805115663088),
 ('сделать', 0.009608853836747316),
 ('тут', 0.009608853836747316),
 ('скамейка', 0.008274798124656415),
 ('друг', 0.007092684106848357),
 ('бегемот', 0.006405902557831544),
 ('найти', 0.006405902557831544),
 ('закрасить', 0.006405902557831544),
 ('наконец', 0.006405902557831544),
 ('вдруг', 0.006405902557831544),
 ('потому', 0.006405902557831544),
 ('потом', 0.006405902557831544)]

In [76]:
# Как это было
collections.Counter(tfidfB).most_common(15)

[('вспоминать', 0.01815888080443157),
 ('начать', 0.01815888080443157),
 ('залезть', 0.01815888080443157),
 ('дождь', 0.01815888080443157),
 ('ошибочно', 0.009079440402215784),
 ('чуть', 0.009079440402215784),
 ('затем', 0.009079440402215784),
 ('папа', 0.009079440402215784),
 ('или', 0.009079440402215784),
 ('иметь', 0.009079440402215784),
 ('произойти', 0.009079440402215784),
 ('полить', 0.009079440402215784),
 ('именно', 0.009079440402215784),
 ('арбуз', 0.009079440402215784),
 ('полёт', 0.009079440402215784)]

In [77]:
# Биография зонтика
collections.Counter(tfidfC).most_common(15)

[('зонтик', 0.01981099876749989),
 ('следующий', 0.014314166627597522),
 ('старый', 0.014314166627597522),
 ('кадр', 0.014314166627597522),
 ('насос', 0.014314166627597522),
 ('с', 0.011886599260499934),
 ('вода', 0.01073562497069814),
 ('шланг', 0.01073562497069814),
 ('заплатка', 0.01073562497069814),
 ('перед', 0.01073562497069814),
 ('зонт', 0.01073562497069814),
 ('струя', 0.01073562497069814),
 ('появляться', 0.01073562497069814),
 ('свой', 0.009245132758166614),
 ('накачать', 0.007157083313798761)]

На самом деле есть библиотека для расчёта tfidf в одну строку. Но об этом и другом в следующих сериях.